# Midterm 2 2023

## FINM 36700 - 2023

### Brian Wickman

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file has sheets for...
* `info` - names and descriptions of each factor
* `factors (excess returns)` - excess returns on several factors
* `portfolios (excess returns)` - excess returns on industry portfolios
* `risk-free rate` - risk-free rates over time

Note the data is **monthly** so any annualizations should use `12` months in a year.

In [115]:
# Set-up & Import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")
sns.set_theme()

# import helper functions
import importlib
import helper_funcs

# Import data
FILEIN = 'data/midterm_2_practice_data.xlsx'

factors = pd.read_excel(FILEIN, sheet_name='factors (excess returns)').set_index('Date')
portfolios = pd.read_excel(FILEIN, sheet_name='portfolios (excess returns)').set_index('Date')
rf_rate = pd.read_excel(FILEIN, sheet_name='risk-free rate').set_index('Date')

### Notation
(Hidden LaTeX commands)

$$\newcommand{\betamkt}{\beta^{i,\text{MKT}}}$$
$$\newcommand{\betahml}{\beta^{i,\text{HML}}}$$
$$\newcommand{\betaumd}{\beta^{i,\text{UMD}}}$$
$$\newcommand{\Eri}{E\left[\tilde{r}^{i}\right]}$$
$$\newcommand{\Emkt}{E\left[\tilde{r}^{\text{MKT}}\right]}$$
$$\newcommand{\Ehml}{E\left[\tilde{r}^{\text{HML}}\right]}$$
$$\newcommand{\Eumd}{E\left[\tilde{r}^{\text{UMD}}\right]}$$

# 1. Short Answer

## 1.

Suppose that we find a set of factors that perfectly hedge any asset. Will these factors work as a linear factor pricing model? 

## 2.

If the Fama-French 3-factor model fit perfectly, would the Treynor ratio be equal for every asset?

## 3.

Suppose the CAPM fits perfectly. Then assets which have higher time-series r-squared metrics on the market factor will have higher Sharpe ratios.

## 4.

Based on the case, what are two ways DFA hopes to generate attractive returns for investors?

## 5.

We analyzed a strategy similar to "AQR's Momentum Funds" (mutual funds.) We found this implementation had much higher returns than the momentum factor of Fama French. What was a major drawback to this construction?

## 6.

From our analysis, momentum has had a negative mean return since 2009. Is this evidence against momentum as a pricing factor? Explain why this is a problem or why it is not a problem.

***

# 2. Linear Factor Pricing Models (LFPMs)

This problem tests the following LFPM:

$$\begin{align}
\Eri = \betamkt \Emkt + \betahml \Ehml + \betaumd \Eumd
\end{align}$$

## 1.

### (8 pts)

Estimate the **time-series (TS)** test of this pricing model. 

For each asset, report the following statistics:
* annualized alpha
* betas
* r-squared

In [90]:
importlib.reload(helper_funcs)

<module 'helper_funcs' from 'C:\\Users\\bwick\\Documents\\PortfolioRiskManagement\\Midterm2\\helper_funcs.py'>

In [57]:
# CAPM TS Regressions
# Apply function to each column in 'portfolios' dataframe
capm_ts_results = pd.DataFrame()
for col in portfolios.columns:
    # Process the column
    temp_df = helper_funcs.capm_ts_regression(portfolios[col], factor = factors)
    # Concatenate the result to the final DataFrame
    capm_ts_results = pd.concat([capm_ts_results, temp_df])

display(capm_ts_results)


,alpha,MKT,HML,UMD,r-squared
NoDur,0.029253,0.739522,0.204580,0.049333,0.6179
Durbl,0.010734,1.271865,0.173595,-0.320023,0.6135
Manuf,-0.000996,1.049482,0.197462,-0.036704,0.8703
Enrgy,-0.015117,0.992222,0.637006,0.075170,0.4656
HiTec,0.028207,1.154959,-0.637135,-0.140638,0.8295
Telcm,0.003506,0.837326,0.094363,-0.084518,0.5881
Shops,0.026739,0.946928,-0.042222,-0.015005,0.7422
Hlth,0.031862,0.757605,-0.119928,0.074058,0.5805
Utils,0.013710,0.527879,0.353033,0.108622,0.3427
Other,-0.019780,1.115433,0.426753,-0.048678,0.9101


## 2.

### (7pts)

Estimate the **cross-sectional (CS)** test of the pricing model. 

Include an intercept in your cross-sectional test.

Report the
* annualized intercept
* annualized regression coefficients
* r-squared

In [92]:
# Cross-sectional test of pricing model
# Cross-sectional CAPM regression (excess returns ~ beta)
capm_cs_results = helper_funcs.capm_cs_regression(portfolios, capm_ts_results)
#display(capm_cs_results.summary())

# Beta's annualized
annualization = 12
print(f"R-squared: {capm_cs_results.rsquared.round(4)}\n"
      f"Intercept: {(capm_cs_results.params[0] * annualization).round(4)}\n"
      f"MKT coefficient: {(capm_cs_results.params[1] * annualization).round(4)}\n"
      f"HML coefficient: {(capm_cs_results.params[2] * annualization).round(4)}\n"
      f"UMD coefficient: {(capm_cs_results.params[3] * annualization).round(4)}")


R-squared: 0.3662
Intercept: 0.7646
MKT coefficient: 0.3839
HML coefficient: -0.1892
UMD coefficient: 0.3636


## 3.

Report the annualized factor premia (expected excess returns of the three factors) as implied by each of the TS and CS estimations.

In [97]:
factor_premia = pd.DataFrame(index = ['TS','CS'], columns = ['MKT Premia','HML Premia','UMD Premia'])
factor_premia.loc['TS',:] = (factors.mean()*12).values # from time-series of factors
factor_premia.loc['CS',:] = [capm_cs_results.params[1], capm_cs_results.params[2], capm_cs_results.params[3]] # from coefficient estimates
display(factor_premia)

,MKT Premia,HML Premia,UMD Premia
TS,0.083853,0.025028,0.061692
CS,0.031992,-0.015767,0.030301


## 4.

Use the r-squared statistics from the TS and CS tests above to assess whether these factors are effective for decomposition and/or pricing.

Be specific as to how the r-squared statistics from the TS and CS tests impact your conclusions.


Nothing could be said about the R-Squared for the time-series case as in the TimeSeries test we do not care about high R-Squared. Thus, the average R-Squared statistic would be unrestricted. For the Cross-sectional test, we want ideally the R-squared to be 1.0, as we expect the factors to explain all of the expected returns of the portfolios. Since the r-squared in CS is not 1.0, there are some other factors that need to be included to perfectly explain the returns for all the assets.

## 5.

Report the annualized pricing mean absolute error (MAE) implied by each of the TS and CS estimations.

In [95]:
ts_mae = capm_ts_results['alpha'].abs().mean().round(4) # MAE of time-series alphas
cs_mae = abs(model.resid).mean() # MAE of cross-sectional residuals
print(f'Time-series MAE: {ts_mae}, Cross-section MAE: {round(cs_mae,4)}')

Time-series MAE: 0.018, Cross-section MAE: 0.0079


## 6.

Which asset has the highest premium as implied by the TS estimation? And as implied by the CS estimation? (For the latter, feel free to include the cross-sectional intercept.)

In [98]:
# Time-series approach
# Step 1: Calculate mean factor premia, annualize it 
# Step 2: Multiply by coefficients from TS regression (exposure of portfolios to each factor)
ts_premium = ((factors.mean() * 12) * capm_ts_results[['MKT', 'HML', 'UMD']]).sum(axis = 1).to_frame('ts_premium').sort_values(by = 'ts_premium', ascending = False)
print(f'{ts_premium.index[0]} portfolio has the highest premium ({ts_premium.iloc[0,0].round(4)}) as implied by the TS estimation.')


Enrgy portfolio has the highest premium (0.1038) as implied by the TS estimation.


In [105]:
# Cross-sectional approach (highest point on graph)
cs_betas = capm_cs_results.params[1:4]
# Step 1: Time-series betas on FCTRS (exposure to each factor) * cross-sectional betas (risk premia of each factor)
cs_premium = (capm_ts_results[['MKT', 'HML', 'UMD']] * cs_betas).sum(axis = 1).to_frame('cs_premium').sort_values(by = 'cs_premium', ascending = False)
print(f'{cs_premium.index[0]} portfolio has the highest premium ({cs_premium.iloc[0,0].round(4)}) as implied by the CS estimation w/o Intercept.')

HiTec portfolio has the highest premium (0.0427) as implied by the CS estimation w/o Intercept.


***

# 3. Additional Analysis

## 1. 

Consider the three-factor pricing model above. How can we assess whether all three factors are useful in this pricing model? 

Specifically, discuss whether the previously estimated regression betas would be informative. If not, what other statistic could we calculate?

## 2.

Suppose we are testing the 3-factor model above, and now we want to allow for time-varying betas.

How could we test the model while allowing for this?

Be specific about the number of regressions we would run and the nature of these regressions.

## 3.

State one advantage and one disadvantage of using the CS estimation as opposed to the TS estimation in fitting the LFPM to the data.

## 4.

Suppose we are investing in just the assets included in our data set. We want to implement a momentum strategy.

Relative to the momentum strategies we studied, do you expect this strategy would have higher or lower...
* mean
* volatility

Explain.

***

# 4. Returns Over Time

## 1.

If Barnstable’s assumptions hold, (log iid returns, normally distributed,) then in what sense is an investment safer in the long-run? And in what sense is it riskier in the long-run?

## 2. 

### (10pts)

Data 
* Make use of the `risk-free rate` tab.
* Construct the **total** factor returns by adding the risk-free rate to the excess `MKT` and `HML` factor returns.

Assumptions
* The total returns are lognormally distributed and iid. 

Report the probability that `MKT` will outperform `HML` over the following 5 years.

In [119]:
# Calculate total, log return
ret_data = pd.concat([factors[['MKT', 'HML']], rf_rate['RF']], axis = 1)
ret_data = ret_data.apply(lambda x: x + ret_data['RF'] if x.name != 'RF' else x)
ret_data = ret_data.drop(columns='RF')
log_ret_data = ret_data.apply(lambda x: np.log(1+x))
log_ret_data.head()

# Probability that MKT will outperform HML in next 5 years
spread = log_ret_data['MKT'] - log_ret_data['HML']
mu, sigma = spread.mean() * 12, spread.std() * np.sqrt(12)
prob = norm.cdf(np.sqrt(5) * mu / sigma)
print(f'Probability that MKT will outperform HML in next 5-years: {round(prob,4)}')

Probability that MKT will outperform HML in next 5-years: 0.7107


***